# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet'])

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import make_multilabel_classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import make_scorer, accuracy_score, f1_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /home/freemo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/freemo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("DisasterResponse", engine)
X = df['message']
#drop also "related" column as there were records with 3 choices
y = df.drop(['id', 'message', 'original', 'genre', 'related'], axis = 1)
categories = y.columns

In [4]:
y.isnull().values.any()

False

In [5]:
categories.value_counts().sum()

35

In [6]:
# check the shape
X.shape
y.shape

(26216, 35)

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

Check randomly the two arrays

### 2. Write a tokenization function to process your text data

In [8]:
# create a tokenization function
def tokenize(text):
    """Tokenize and clean text.
    
    Args: message text
    
    Returns: normalized tokens
    """
    
    # remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [9]:
# test function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

https://medium.com/all-things-ai/in-depth-parameter-tuning-for-random-forest-d67bb7e920d

In [10]:
# build the pipeline, parameters chosen after several attempts to maximize processing speed
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, oob_score=True, 
                                                         n_jobs=-1,random_state=50, max_features="auto", 
                                                         min_samples_leaf=50)))
])

In [ ]:
# RandomForestRegressor(n_estimator = 100, oob_score = TRUE, n_jobs = -1,random_state =50, 
# max_features = "auto", min_samples_leaf = 50)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# stratify=y,test_size=0.25

In [11]:
# split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

In [12]:
# train classifier, around 8 mins to complete / it took just 3mins with the new settings
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fb57b26f7a0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=50,
                                                                        n_jobs=-1,
                                                                        oob_score=True,
                                                                        random_state=50)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [14]:
# check metrics
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6515
           1       0.00      0.00      0.00      1350

    accuracy                           0.83      7865
   macro avg       0.41      0.50      0.45      7865
weighted avg       0.69      0.83      0.75      7865

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7827
           1       0.00      0.00      0.00        38

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.64      0.99      0.78      4615
           1 

/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7752
           1       0.00      0.00      0.00       113

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.99      0.98      7865

------------------------------------------------------

FEATURE: money

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7694
           1       0.00      0.00      0.00       171

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.96      0.98      0.97      7865

------------------------------------------------------

FEATURE: missing_people

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7771
           1       0.00      0.00      0.00        94

    accuracy                 

In [14]:
#print(classification_report(y_test, y_pred, target_names=y.columns))
# multilabel_confusion_matrix(y_test, y_pred)

In [15]:
# check data
from collections import Counter
Counter(y_train)
Counter(y_test)

Counter({'request': 1,
         'offer': 1,
         'aid_related': 1,
         'medical_help': 1,
         'medical_products': 1,
         'search_and_rescue': 1,
         'security': 1,
         'military': 1,
         'child_alone': 1,
         'water': 1,
         'food': 1,
         'shelter': 1,
         'clothing': 1,
         'money': 1,
         'missing_people': 1,
         'refugees': 1,
         'death': 1,
         'other_aid': 1,
         'infrastructure_related': 1,
         'transport': 1,
         'buildings': 1,
         'electricity': 1,
         'tools': 1,
         'hospitals': 1,
         'shops': 1,
         'aid_centers': 1,
         'other_infrastructure': 1,
         'weather_related': 1,
         'floods': 1,
         'storm': 1,
         'fire': 1,
         'earthquake': 1,
         'cold': 1,
         'other_weather': 1,
         'direct_report': 1})

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
# get params 
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [17]:
# check all features
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fb57b26f7a0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=50,
                                                          n_jobs=-1,
                                                          oob_score=True,
                                                          random_state=50)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fb57b26f7a0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=50,
                                                        n_jobs=-1,
                                                        oob_score=True,
                                                        random_state=50)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encodi


parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"]
    
             }

In [16]:
# working model
# parameters = {'clf__estimator__n_estimators': [100, 200]}

# rf_Grid.best_params_

#parameters = {'tfidf__norm': ['l1','l2'],'clf__estimator__criterion': ["gini", "entropy"]}

parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

In [18]:
# build GridSearch with parameters
parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"]
    
             }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3)

In [19]:
# nothing for 15min, with improved settings it took just 2mins
cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  1.3min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  1.3min remaining:   45.8s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.3min finished


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fb57b26f7a0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=50,
                                                                                               n_jobs=-1,
                                                                                               oob_score=True,
                                                                                               random_state=50)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'tfidf__norm': ['l1', 'l2']},
             verbose=3)

In [20]:
# check the best parameters
cv.best_params_

{'clf__estimator__criterion': 'gini', 'tfidf__norm': 'l2'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# test model
y_pred = cv.predict(X_test)

In [23]:
# check metrics
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6515
           1       0.00      0.00      0.00      1350

    accuracy                           0.83      7865
   macro avg       0.41      0.50      0.45      7865
weighted avg       0.69      0.83      0.75      7865

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7827
           1       0.00      0.00      0.00        38

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.64      0.99      0.78      4615
           1 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# skipped that, it took too much time to train new models

### 9. Export your model as a pickle file

In [26]:
import pickle
pickle.dump(pipeline, open('models/classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.